In [12]:
import numpy as np
import pandas as pd

In [13]:
#bgvar_16 = pd.read_stata('avars_201601_EN_1.0p.dta')
hea_14 = pd.read_stata('ch16i_EN_1.0p.dta')
rel_14 = pd.read_stata('cr16i_1.0p_EN.dta')
soc_14 = pd.read_stata('cs16i_EN_1.0p.dta')
fam_14 = pd.read_stata('cf16i_EN_1.0p.dta')
wor_14 = pd.read_stata('cw16i_EN_2.0p.dta')
per_14 = pd.read_stata('cp17i_EN_1.0p.dta')
val_14 = pd.read_stata('cv16h_EN_1.0p.dta')
ass_14 = pd.read_stata('ca16e_EN_1.0p.dta')
inc_14 = pd.read_stata('ci16i_EN_3.0p.dta')
hou_14 = pd.read_stata('cd16i_EN_1.0p.dta')

In [14]:
#column combinators (replaces given columns with one containing their avg values)
def average_columns(df, *columns):
    new_column_name = columns[-1]
    
    try:
        selected_columns = list(columns[:-1])
        avg_col = df[selected_columns].astype(float).mean(axis=1)
        return pd.DataFrame({new_column_name: avg_col})
    except KeyError as e:
        print(f'KeyError: {e} not found in DataFrame columns.')
        return pd.DataFrame({new_column_name: np.nan})

#replaces given columns with one containing their sums (not working ;-())
def sum_columns(df, new_column, *columns):
    df[new_column] = df[columns].sum(axis=1)

Background variables:

1. geslacht to sex
2. leeftijd to age
3. lftdcat to age_cat
4. aantalhh to hhm_num
5. aantalki to hh_chi
6. burgstat to civ_sta
7. woning to dwe_typ
8. sted to urb_lvl (Urban character: Surrounding address density per km2 extremely urban 2,500 or more very 1,500 to 2,500 moderately 1,000 to 1,500 slightly 500 to 1,000 not less than 500) 
9. belbezig to pri_occ 
10. brutoink to mon_inc 
11. brutocat mon_inc_cat 12. brutohh_f to hh_inc 
13. oplzon to edu_lvl

In [15]:
'''bgvar_14[bgvar_14.select_dtypes(include='category').columns] = bgvar_14[bgvar_14.select_dtypes(include='category').columns].apply(lambda col: col.astype('category').cat.codes)
bg_var_names = {'nomem_encr':'nomem_encr', 'geslacht':'sex','leeftijd':'age', 'lftdcat':'age_cat', 'aantalhh':'hhm_num', 'aantalki':'hh_chi', 'burgstat':'civ_sta', 'woning':'dwe_typ', 'sted':'urb_lvl',  'belbezig':'pri_occ', 'brutoink':'mon_inc', 'brutocat':'mon_inc_cat','brutohh_f':'hh_inc', 'oplzon':'edu_lvl'}
bg_var = bgvar_14.loc[:, list(bg_var_names.keys())].rename(columns=bg_var_names)
bg_var.head()
'''

,nomem_encr,sex,age,age_cat,hhm_num,hh_chi,civ_sta,dwe_typ,urb_lvl,pri_occ,mon_inc,mon_inc_cat,hh_inc,edu_lvl
0,800009.0,0,60.0,5,1,0,0,0,4,10,0,13,5623.422785,4
1,800012.0,1,45.0,4,2,1,2,0,3,0,487,4,6015.000000,1
2,800015.0,0,53.0,4,3,2,0,0,0,0,949,7,3469.000000,5
3,800033.0,0,24.0,1,3,3,4,1,3,6,80,2,NaN,4
4,800042.0,1,40.0,3,3,2,0,0,2,7,36,1,2685.000000,2


Concepts in Health:
1. **Subjective health** (ch16i004 to sub_hea)
2. **Relative health** (ch16i005 to rel_hea)
3. Life expectancy
4. **Mental health** (all neg as men_hea_neg and and all pos as men_hea_pos)
5. Height, weight
6. **Chronical problems** (ch16i018 to disability)
7. Health problems
8. Impediment in labour
9. **Substance use** (ch16i126 to smoker, ch16i133 to alcohol, all substances to subs)
10. Medication
11. Physical activity - only until 2018
12. Health improving behaviour - - only until 2018
13. Use of healthcare
14. Sight and hearing
15. Health insurance

In [36]:
#select categorical columns and convert to numeric
hea_14[hea_14.select_dtypes(include='category').columns] = hea_14[hea_14.select_dtypes(include='category').columns].apply(lambda col: col.astype('category').cat.codes)

#average for mental health and drugs
hea_toavg = [('ch16i011','ch16i014','ch16i012', 'men_hea_neg'),('ch16i013', 'ch16i015', 'men_hea_pos'), ('ch16i159', 'ch16i160', 'ch16i161', 'ch16i161', 'ch16i162', 'ch16i163', 'subs')]
hea_14 = pd.concat([hea_14] + [average_columns(hea_14, *column_set) for column_set in hea_toavg], axis=1)

#create a new df hea_14_con with selected columnns and new columns containing avgs (representing concepts)
hea_con = {'nomem_encr':'nomem_encr', 'ch16i004': 'sub_hea', 'ch16i005': 'rel_hea', 'ch16i018': 'disability', 'ch16i020':'mob_issues', 'ch16i126':'smoker', 'ch16i133':'alcohol', 'men_hea_neg':'men_hea_neg', 'men_hea_pos':'men_hea_pos', 'subs':'subs'}
hea_14_con = hea_14.loc[:, list(hea_con.keys())].rename(columns=hea_con)

#ready
hea_14_con.head()

,nomem_encr,sub_hea,rel_hea,disability,mob_issues,smoker,alcohol,men_hea_neg,men_hea_neg,men_hea_pos,men_hea_pos,subs,subs
0,800009.0,2,2,0,0,1,1,0.000000,0.000000,2.5,2.5,0.0,0.0
1,800015.0,2,2,0,0,-1,2,0.000000,0.000000,4.0,4.0,0.0,0.0
2,800042.0,2,2,1,1,1,3,0.333333,0.333333,3.0,3.0,0.0,0.0
3,800054.0,2,2,1,0,-1,1,0.000000,0.000000,4.0,4.0,0.0,0.0
4,800057.0,3,3,1,0,1,1,0.000000,0.000000,4.0,4.0,0.0,0.0


Concepts in Religion and Ethnicity:

1. **Religious upbringing** (cr16i135 to rel_upb)
2. **Religious affiliation** (cr16i144 to rel_aff)
3. **Religiosity** (cr16i162 to relig)
4. Religious orthodoxy
5. Nationality 
6. **Origin** (cr16i079 – cr16i087 to lang_ori, cr16i079 indicating Dutch or not)
7. **Ethnic identification** (cr16i164 to eth_id)
8. Language proficiency and use

In [39]:
rel_con = {'nomem_encr':'nomem_encr', 'cr16i079':'lang_ori'}
rel_14[rel_14.select_dtypes(include='category').columns] = rel_14[rel_14.select_dtypes(include='category').columns].apply(lambda col: col.astype('category').cat.codes)
rel_14_con = rel_14.loc[:, list(rel_con.keys())].rename(columns=rel_con)
rel_14_con.head()

,nomem_encr,lang_ori
0,800009.0,1
1,800015.0,0
2,800042.0,1
3,800054.0,1
4,800057.0,1


Concepts in Social Intergation and Leisure:

1. Loneliness
2. Social contacts 
3. **Satisfaction leisure** (cs16i001 to lei_sat)
4. Social engagement
5. Volunteer aid
6. Time expenditure voluntary work
7. **Cultural participation** (sum cs16i494 - cs16i496 to cul_par)
8. **Holidays** (cs16i103 to ab_hol)
9. **Sport** (cs16i105 to spo_h)
10. Media usage

In [41]:
soc_14[soc_14.select_dtypes(include='category').columns] = soc_14[soc_14.select_dtypes(include='category').columns].apply(lambda col: col.astype('category').cat.codes)
cultosum = ['cs16i494', 'cs16i495', 'cs16i517', 'cs16i093', 'cs16i094', 'cs16i516', 'cs16i496']
soc_14['cul_sum'] = soc_14[cultosum].sum(axis=1)
soc_con = {'nomem_encr':'nomem_encr','cs16i001':'lei_sat', 'cul_sum':'cul_par', 'cs16i103':'ab_hol', 'cs16i105':'spo_h'}
soc_14_con = soc_14.loc[:, list(soc_con.keys())].rename(columns=soc_con)
print(soc_14_con)

      nomem_encr  lei_sat  cul_par  ab_hol  spo_h
0       800009.0        8       -4       2    NaN
1       800015.0        8       -7       3    NaN
2       800042.0        7       -7       1    5.0
3       800054.0       10       -6       0    NaN
4       800057.0        7       -3       3    2.0
...          ...      ...      ...     ...    ...
5553    899905.0       10       -6       1    3.0
5554    899914.0       10       -7       2    NaN
5555    899923.0        6       -4       1    6.0
5556    899954.0        8       -7       0    NaN
5557    899960.0        7       -5       2    1.5

[5558 rows x 5 columns]


Concepts in Family and Household:

1. **Parental relations** (cf16i145+cf16i146 to par_int)
2. **Partnership** (cf16i025 to liv_tog)
3. Marriage
4. Marital history
5. **Children** (cf16i455 to chi_num, cf16i456 to chi1_bir)
6. **Partnership and childbirth intentions** (cf16i128 to chi_fut)
7. Infertility
8. Parental support
9. Parent support
10. Quality of relationship
11. Division of domestic tasks
12. Perceived burden domestic tasks
13. Perceived burden child care
14. Perceived fairness division tasks
15. Child education: Supervision
16. Childcare

In [42]:
fam_16[fam_16.select_dtypes(include='category').columns] = fam_16[fam_16.select_dtypes(include='category').columns].apply(lambda col: col.astype('category').cat.codes)
famtosum = ['cf16i143', 'cf16i144']
fam_16['fam_sum'] = fam_16[famtosum].sum(axis=1)
fam_con = {'nomem_encr':'nomem_encr', 'fam_sum':'par_int', 'cf16i025':'liv_tog', 'cf16i455':'chi_num', 'cf16i456':'chi1_bir', 'cf16i128':'chi_fut'}
fam_16_con = fam_16.loc[:, list(fam_con.keys())].rename(columns=fam_con)
print(fam_16_con)

      nomem_encr  par_int  liv_tog  chi_num  chi1_bir  chi_fut
0       800009.0       -2        0      6.0    1981.0       -1
1       800015.0        1        0      2.0    1994.0       -1
2       800042.0        4        0      2.0    2005.0        1
3       800054.0       -2       -1      2.0    1971.0       -1
4       800057.0        4        0      3.0    2007.0        1
...          ...      ...      ...      ...       ...      ...
5522    899914.0       -2        0      2.0    1978.0       -1
5523    899923.0        4        0      2.0    1993.0       -1
5524    899954.0       -2       -1      NaN       NaN       -1
5525    899960.0        2        0      NaN       NaN        0
5526    899993.0        0       -1      NaN       NaN        2

[5527 rows x 6 columns]


Concepts in Work and Schooling:

1. Employment / activity status
2. Hours of paid work
3. **Characteristics current or last job** (cw16i136 to wor_min, cw16i140 to wor_nig, cw16i142 to wor_hom, cw16i402 to wor_sect)
4. Average pay job
5. Second jobs
6. Work aspirations
7. **Satisfaction** (cw16i128 to inc_sat)
8. Employment conditions
9. Commuting
10. Pensions
11. Non-working, non-pension
12. Job search
13. Satisfaction education
14. Highest qualification
15. Highest education
16. Qualifications for job
17. Education and training in last year

cw16i439 to chi_<8

In [43]:
wor_16[wor_16.select_dtypes(include='category').columns] = wor_16[wor_16.select_dtypes(include='category').columns].apply(lambda col: col.astype('category').cat.codes)
wor_con = {'nomem_encr':'nomem_encr', 'cw16i136':'wor_min', 'cw16i140':'wor_nig', 'cw16i142':'wor_hom', 'cw16i402':'wor_sect', 'cw16i128':'inc_sat', 'cw16i439':'chi_<8'}
wor_16_con = wor_16.loc[:, list(wor_con.keys())].rename(columns=wor_con)
print(wor_16_con)

      nomem_encr  wor_min  wor_nig  wor_hom  wor_sect  inc_sat  chi_<8
0       800009.0      NaN       -1       -1        -1       -1      -1
1       800015.0      5.0        0        0        10        7       1
2       800042.0     10.0        0        0        13        8       1
3       800054.0      NaN       -1       -1        -1       -1      -1
4       800057.0     30.0        1        0         9        7       0
...          ...      ...      ...      ...       ...      ...     ...
5827    899826.0      NaN       -1       -1        -1       -1      -1
5828    899914.0      NaN       -1       -1        -1       -1      -1
5829    899923.0     20.0        0        3        10        7       1
5830    899954.0      NaN       -1       -1        -1       -1      -1
5831    899960.0     35.0        0        0         9        7      -1

[5832 rows x 7 columns]


Concepts in Personality:
1. Happiness
2. **Life satisfaction** (cp16i014-016 to life_satf, cp16i070-079 as self_est_pos or self_est_neg)
3. Big five personality
4. Survey attitude
5. Self esteem
6. Mood
7. Need to evaluate
8. **Value orientation** (cp16i100 to res, cp16i110 to ind, cp16i118 to fam_sec, cp16i119 to free, cp16i129 to comf_life, cp16i133 to soc_rec)
9. Inclusion of Others in the Self
10. Social desirability
11. Affects
12. Need for Cognition
13. **Social trust** (cp16i019 to trust )

Concepts in Values:
1. Beliefs about maternal employment
2. **Gender role attitudes** (cv16i113 to gend_roles)
3. Political position ethnic minorities *page not working*
4. Right wing attitudes *page not working*
5. **Attitude towards foreigners** (cv16i120 to nationalist)
6. Marriage attitudes
7. Family solidarity
8. Norm of having children
9. Work ethic
10. Conservatism
11. Economic conservatism

Concepts in Economic Situtation: Assets:
1. **Assets** (ca16i008 to ass_own, ca16i023 to ass_val, ca16i012 to tot_bal)

In [29]:
ass_16['ca16e023'] = pd.to_numeric(ass_16['ca16e023'], errors='coerce')
ass_16[ass_16.select_dtypes(include='category').columns] = ass_16[ass_16.select_dtypes(include='category').columns].apply(lambda col: col.astype('category').cat.codes)
ass_con = {'nomem_encr':'nomem_encr', 'ca16e008':'ass_own','ca16e023':'ass_val', 'ca16e012':'tot_bal'}
ass_16_con = ass_16.loc[:, list(ass_con.keys())].rename(columns=ass_con)

ass_16_con.head()

,nomem_encr,ass_own,ass_val,tot_bal
0,800009.0,1,1.000000e+04,3.450000e+03
1,800015.0,0,NaN,3.100000e+04
2,800042.0,1,1.000000e+10,1.000000e+10
3,800054.0,0,NaN,1.000000e+10
4,800057.0,1,2.500000e+04,3.000000e+04


Concepts in Income: 
1. Subjective standard of living
2. Satisfaction income
3. Employment income / Earnings
4. Self-employment income
5. Non-employment income: pension
6. Non-employment income: benefits
7. Non-employment income: other
8. Credit, debt, payments
9. **Total income** (ci16i226 to income)
10. **Financial/material well-being** (ci16i252 to fin_sit)
11. Financial expectations
12. Financial management
13. Material deprivation


In [30]:
inc_16[inc_16.select_dtypes(include='category').columns] = inc_16[inc_16.select_dtypes(include='category').columns].apply(lambda col: col.astype('category').cat.codes)

#create a new df per_22_con with selected columnns and new columns containing avgs (representing concepts)
inc_con = {'nomem_encr':'nomem_encr','ci16i226': 'income', 'ci16i252': 'fin_sit' }
inc_16_con = inc_16.loc[:, list(inc_con.keys())].rename(columns=inc_con)

#ready
inc_16_con.head()

,nomem_encr,income,fin_sit
0,800009.0,-1,3
1,800015.0,-1,3
2,800042.0,2,3
3,800054.0,11,2
4,800057.0,9,3


Concepts for Housing:
1. Satisfaction housing
2. Housing characteristics
3. Housing expenditure 
4. **Housing wealth** (cd22o025 to pur_price)

In [31]:
hou_16[hou_16.select_dtypes(include='category').columns] = hou_16[hou_16.select_dtypes(include='category').columns].apply(lambda col: col.astype('category').cat.codes)

hou_con = {'nomem_encr':'nomem_encr','cd16i025': 'pur_price'}
hou_16_con = hou_16.loc[:, list(hou_con.keys())].rename(columns=hou_con)
hou_16_con.head()

,nomem_encr,pur_price
0,800015.0,94463.0
1,800054.0,NaN
2,800057.0,605000.0
3,800085.0,NaN
4,800170.0,120000.0


In [33]:
#merge
all_16 = [bg_var, fam_16_con, hea_16_con, soc_16_con, val_16_con, wor_16_con, inc_16_con, ass_16_con, hou_16_con]

merged_16 = all_16[0]

for all in all_16[1:]:
    merged_16 = pd.merge(merged_16, all, on='nomem_encr', how='inner')

In [34]:
merged_16.head()

,nomem_encr,sex,age,age_cat,hhm_num,hh_chi,civ_sta,dwe_typ,urb_lvl,pri_occ,...,wor_hom,wor_sect,inc_sat,chi_<8,income,fin_sit,ass_own,ass_val,tot_bal,pur_price
0,800015.0,0,53.0,4,3,2,0,0,0,0,...,0,10,7,1,-1,3,0,NaN,3.100000e+04,94463.0
1,800054.0,1,69.0,6,0,0,2,1,3,8,...,-1,-1,-1,-1,11,2,0,NaN,1.000000e+10,NaN
2,800057.0,0,41.0,3,4,3,4,0,0,0,...,0,9,7,0,9,3,1,25000.0,3.000000e+04,605000.0
3,800170.0,1,57.0,5,3,3,2,0,3,0,...,0,12,2,1,-1,3,0,NaN,3.500000e+04,120000.0
4,800186.0,1,76.0,6,1,0,3,0,3,7,...,-1,-1,-1,-1,5,3,1,2000.0,1.000000e+10,29000.0
